In [1]:
from midrc_dataset import midrc_SIMCLR_dataset
import torchvision.transforms as transforms

root_dir = '../data/resized_224X224'

json_file = '../data/label_info/refined_study_wise_images.json'

annotations_file = 'MIDRC mRALE Mastermind Training Annotations_2079_20230428.csv'

transform = transforms.Compose(transforms.PILToTensor())

dataset = midrc_SIMCLR_dataset(root_dir, annotations_file, json_file, transform=transform)

img1,img2,score1, score2 = dataset[0]

img1

AttributeError: 'midrc_SIMCLR_dataset' object has no attribute 'data_dir'

# Train

# Save model Weights